In [1]:
# Instal dependency
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-184bx_e5

  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-184bx_e5

  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1

  Preparing metadata (setup.py) ... done





















   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 2.3 MB/s eta 0:00:00



  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=2f1a327d4dd06d78b957a76201b5362dcd788bc42b4764a595edcf4e06524025

  Stored in directory: /tmp/pip-ephem-wheel-cache-e9apajqf/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4

Successfully built clip




In [2]:
# Import module
import os
import clip
import glob
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm

# Parse data path

In [3]:
keyframes_dir_list = [f'/kaggle/input/{x}/Keyframes' for x in os.listdir('/kaggle/input')]
all_keyframe_paths = dict()
for keyframe_dir in keyframes_dir_list:
    for part in sorted(os.listdir(keyframe_dir)):
        data_part = part.split('_')[-2] # L01, L02 for ex
        all_keyframe_paths[data_part] =  dict()
for keyframe_dir in keyframes_dir_list:
    for data_part in sorted(all_keyframe_paths.keys()):
        data_part_path = f'{keyframe_dir}/{data_part}_extra'
        if os.path.isdir(data_part_path):
            video_dirs = sorted(os.listdir(data_part_path))
            video_ids = [video_dir.split('_')[-1] for video_dir in video_dirs]
            for video_id, video_dir in zip(video_ids, video_dirs):
                keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.jpg'))
                all_keyframe_paths[data_part][video_id] = keyframe_paths

# Model

In [4]:
##### Load Model #####
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model, preprocess = clip.load("ViT-B/16", device=device)

cuda


100%|███████████████████████████████████████| 335M/335M [00:09<00:00, 36.4MiB/s]


In [5]:
bs = 512
save_dir = './CLIP_features'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for key, video_keyframe_paths in all_keyframe_paths.items():
    video_ids = sorted(video_keyframe_paths.keys())
    
    if not os.path.exists(os.path.join(save_dir, key)):
        os.mkdir(os.path.join(save_dir, key))
    
    for video_id in tqdm(video_ids):
        video_feats = []
        video_keyframe_path = video_keyframe_paths[video_id]
        for i in range(0, len(video_keyframe_path), bs):
            # Support batchsize inferencing
            images = []
            image_paths = video_keyframe_path[i:i+bs]
            for image_path in image_paths:
                image = preprocess(Image.open(image_path)).unsqueeze(0)
                images.append(image)
            images = torch.cat(images).to(device)

            with torch.no_grad():
                image_feats = model.encode_image(images)
            image_feats /= image_feats.norm(dim=-1, keepdim=True)

            for b in range(image_feats.shape[0]):
                video_feats.append(image_feats[b].detach().cpu().numpy().astype(np.float32).flatten())
        
        np.save(f'{save_dir}/{key}/{video_id}.npy', video_feats)

100%|██████████| 31/31 [18:32<00:00, 35.89s/it]

100%|██████████| 31/31 [22:10<00:00, 42.91s/it]

100%|██████████| 30/30 [19:56<00:00, 39.87s/it]

100%|██████████| 30/30 [22:03<00:00, 44.13s/it]

100%|██████████| 30/30 [19:01<00:00, 38.03s/it]

100%|██████████| 30/30 [20:25<00:00, 40.86s/it]

100%|██████████| 29/29 [20:56<00:00, 43.31s/it]

100%|██████████| 29/29 [22:07<00:00, 45.79s/it]

100%|██████████| 31/31 [19:41<00:00, 38.10s/it]

100%|██████████| 31/31 [20:59<00:00, 40.64s/it]

100%|██████████| 31/31 [20:25<00:00, 39.52s/it]

100%|██████████| 30/30 [21:59<00:00, 43.99s/it]
